In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_7월_형태소분석.txt", encoding='UTF-8')
df['morphs']=df['morphs'].apply(lambda x : x.strip("'][").split("', '"))
print(df.shape, df.columns)
docs = df['morphs'].to_list()
print(len(docs))

(901126, 4) Index(['split_str', 'org_idx', 'morphs', 'pos_tag'], dtype='object')
901126


In [3]:
df

,split_str,org_idx,morphs,pos_tag
0,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...,0,"[25일, 급여, 삼성, 카드, 특판, 적금, 서울, 행복, 신협, 구즉, 신협, ...","[('25일', 'Number'), ('급여', 'Foreign'), ('삼성', ..."
1,근데 급 다음주부터 발령나서 다른 곳으로 출근한다,0,"[근데, 급, 다음주, 부터, 발령, 나서, 다른, 곳, 으로, 출근, 한다]","[('근데', 'Adverb'), ('급', 'Noun'), ('다음주', 'Nou..."
2,2 3일 전에 알려주고 갑자기 거기로 출근이야 좌천인가 걱정됨,0,"[2, 3일, 전, 에, 알려주고, 갑자기, 거기, 로, 출근, 이야, 좌천, 인가...","[('2', 'Number'), ('3일', 'Number'), ('전', 'Nou..."
3,잘 적응하라고 뇌물을 준비해야 하나 ㅋㅋ 갑자기 문자메세지로 삼성카드 5월 링크 이...,0,"[잘, 적응하라고, 뇌물, 을, 준비, 해야, 하나, ㅋㅋ, 갑자기, 문자, 메세지...","[('자다', 'Verb'), ('적응하다', 'Adjective'), ('뇌물',..."
4,소소하게 가계부 적립할게 생겨서 기분 좋은 날 2 화요일 오래전 엄마 집 옆에 있는...,0,"[소소하게, 가계부, 적립, 할게, 생겨서, 기분, 좋은, 날, 2, 화요일, 오래...","[('소소하다', 'Adjective'), ('가계부', 'Noun'), ('적립'..."
...,...,...,...,...
901121,현재 지하 1층에서 흙과몸 성긴 연결 촘촘한 관계 6 9 전시실 를 테마로 강요배 ...,18988,"[현재, 지하, 1, 층, 에서, 흙, 과, 몸, 성, 긴, 연결, 촘촘, 한, 관...","[('현재', 'Noun'), ('지하', 'Noun'), ('1', 'Number..."
901122,그 외에 Big Man 다시 일어서는 몸 박치호 애도 상실의 끝에서 김수자 게르하르...,18988,"[그, 외, 에, Big, Man, 다시, 일어서는, 몸, 박치, 호, 애도, 상실...","[('그', 'Noun'), ('외', 'Noun'), ('에', 'Josa'), ..."
901123,광양시 문화관광 061 797 3 광양시 관광안내소 광양 문화관광 1경 백운산 4대...,18988,"[광양시, 문화, 관광, 061, 797, 3, 광양시, 관광, 안내소, 광양, 문...","[('광양시', 'Noun'), ('문화', 'Noun'), ('관광', 'Noun..."
901124,자막보기 공지사항 제11회 최수종과 함께하 동북아 LNG Hub 광양시립소년소녀합창...,18988,"[자막, 보기, 공지, 사항, 제, 11회, 최수종, 과, 함께하, 동북아, LNG...","[('자막', 'Noun'), ('보기', 'Noun'), ('공지', 'Noun'..."


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
# for i in tqdm(range(len(docs))):
#     docs2[i] = docs2[i].split()

In [5]:
#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("../dataset/2022/본문_형태소분석/7월/나무문화_7월_형태소분석(명사).csv", encoding='UTF-8')
print(topwords_df.shape, topwords_df.columns)
topwords50 = topwords_df['명사'].to_list()[:50]
print(len(topwords50))
print(topwords50)

(151672, 2) Index(['명사', '명사빈도'], dtype='object')
50
['것', '수', '그', '이', '곳', '사람', '때', '등', '우리', '말', '생각', '더', '내', '문화', '나', '시간', '위', '나무', '의', '사진', '길', '안', '중', '때문', '전', '제', '마을', '하나', '일', '여행', '집', '작품', '날', '공간', '위해', '다른', '아이', '시작', '카페', '저', '거', '대한', '층', '개', '속', '한국', '또', '모습', '마음', '및']


In [6]:
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]

100%|██████████| 901126/901126 [00:06<00:00, 130626.30it/s]


In [7]:
#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))

104
95


In [8]:
#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1

100%|██████████| 901126/901126 [10:26<00:00, 1439.33it/s] 


In [9]:
#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(freq_count, orient='index')

In [10]:
#### 딕트to데이터프레임 저장
tt_freq_df.to_csv("../dataset/2022/단어동시출현행렬/7월/단어동시출현_딕셔너리_7월.csv", encoding='UTF-8')

In [11]:
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(tt_freq_df.index[i][0], tt_freq_df.index[i][1], tt_freq_df[0][i]) for i in tqdm(range(len(tt_freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()

100%|██████████| 158755367/158755367 [17:09<00:00, 154159.36it/s]


(158755367, 3)


,term1,term2,freq
0,명동,문화,36
1,문화,의,18151
2,문화,부시,14
3,YOOX,문화,1
4,문화,카카오,49


In [12]:
#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("../dataset/2022/단어동시출현행렬/7월/단어동시출현_df_7월.csv", index=False, encoding='UTF-8')

In [13]:
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3

100%|██████████| 158755367/158755367 [33:02<00:00, 80091.43it/s]


,term1,term2,freq
0,문화,의,18151
1,문화,전,2354
2,문화,카페,1390
3,날,문화,1160
4,문화,일,1754
...,...,...,...
1220,대한,카페,135
1221,모습,문화,787
1222,길,및,561
1223,때문,마을,321


In [14]:
#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("../dataset/2022/단어동시출현행렬/7월/단어동시출현_df_상위50_7월.csv", index=False, encoding='UTF-8')

In [15]:
#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]

(50, 50)


100%|██████████| 1225/1225 [00:00<00:00, 1489.52it/s]


In [16]:
#### 1-mode 매트릭스 확인하기
word_co_matx

,것,수,그,이,곳,사람,때,등,우리,말,생각,더,내,문화,나,시간,위,나무,의,사진,길,안,중,때문,전,제,마을,하나,일,여행,집,작품,날,공간,위해,다른,아이,시작,카페,저,거,대한,층,개,속,한국,또,모습,마음,및
것,0.0,28010.0,27080.0,101436.0,8696.0,15951.0,13443.0,8176.0,11814.0,13218.0,11798.0,10010.0,10959.0,5663.0,13668.0,5918.0,7632.0,4707.0,77203.0,3964.0,3779.0,8052.0,6862.0,8760.0,6379.0,5279.0,1889.0,6050.0,8349.0,2655.0,3945.0,3797.0,3930.0,3264.0,5592.0,6983.0,3720.0,4331.0,1627.0,5046.0,3712.0,6228.0,1678.0,3336.0,6187.0,3166.0,4756.0,3861.0,5301.0,1999.0
수,28010.0,0.0,15270.0,64361.0,11026.0,9785.0,8347.0,9906.0,7586.0,7103.0,6957.0,7312.0,7200.0,6635.0,9660.0,6217.0,5752.0,4910.0,56959.0,3926.0,4318.0,5325.0,5219.0,6199.0,5476.0,4521.0,2549.0,3964.0,5703.0,3152.0,2940.0,3815.0,2891.0,5759.0,3799.0,4541.0,4089.0,2911.0,2450.0,3198.0,2497.0,4279.0,2776.0,3175.0,4628.0,2672.0,3151.0,3818.0,3850.0,3234.0
그,27080.0,15270.0,0.0,56055.0,7399.0,10983.0,9284.0,5126.0,6787.0,8228.0,5910.0,5300.0,6880.0,3509.0,8297.0,3258.0,5291.0,3321.0,54163.0,2276.0,2851.0,5246.0,6212.0,4956.0,4603.0,3512.0,1546.0,3636.0,5023.0,1487.0,2940.0,3032.0,3017.0,1885.0,3612.0,3757.0,2066.0,3005.0,1042.0,2615.0,2456.0,4147.0,1096.0,2269.0,4715.0,2013.0,3236.0,2950.0,3285.0,1393.0
이,101436.0,64361.0,56055.0,0.0,39327.0,39197.0,33912.0,26743.0,23540.0,25661.0,24149.0,22172.0,24506.0,16652.0,31572.0,18318.0,17508.0,16027.0,190172.0,14454.0,13204.0,20878.0,18725.0,18034.0,18943.0,15381.0,8679.0,13434.0,17331.0,8653.0,12146.0,11351.0,14621.0,12335.0,12269.0,14088.0,11232.0,12264.0,8201.0,13724.0,12243.0,12158.0,7751.0,11001.0,13436.0,9242.0,11463.0,12434.0,12555.0,7117.0
곳,8696.0,11026.0,7399.0,39327.0,0.0,4360.0,4061.0,3694.0,2663.0,2091.0,2847.0,2928.0,3128.0,3333.0,3998.0,3014.0,2288.0,2848.0,23074.0,2984.0,3252.0,2813.0,3376.0,1789.0,2684.0,2431.0,2599.0,1857.0,1851.0,3505.0,1929.0,944.0,1762.0,2723.0,1687.0,2339.0,2644.0,1516.0,2530.0,2291.0,1194.0,870.0,1411.0,1602.0,1700.0,1235.0,1574.0,1780.0,1674.0,975.0
사람,15951.0,9785.0,10983.0,39197.0,4360.0,0.0,5231.0,2728.0,4288.0,5454.0,4630.0,3556.0,4386.0,2391.0,6363.0,2375.0,2710.0,1996.0,27581.0,1956.0,1868.0,3531.0,2755.0,2990.0,2359.0,1748.0,1322.0,2102.0,3559.0,1069.0,1873.0,1157.0,1790.0,1290.0,2304.0,3637.0,1139.0,1630.0,957.0,2118.0,1896.0,1853.0,734.0,1368.0,2398.0,1212.0,1887.0,1786.0,2564.0,641.0
때,13443.0,8347.0,9284.0,33912.0,4061.0,5231.0,0.0,2876.0,3897.0,4063.0,3817.0,3308.0,4391.0,1732.0,5665.0,2466.0,2323.0,2012.0,23637.0,1923.0,1677.0,3456.0,2665.0,2349.0,2762.0,2407.0,839.0,1969.0,2910.0,1526.0,2081.0,1012.0,2033.0,911.0,1812.0,2076.0,1548.0,2117.0,783.0,2047.0,1902.0,1508.0,766.0,1327.0,1838.0,1161.0,1780.0,1587.0,1937.0,622.0
등,8176.0,9906.0,5126.0,26743.0,3694.0,2728.0,2876.0,0.0,1727.0,2148.0,1390.0,1649.0,2388.0,4016.0,3859.0,1689.0,3101.0,2497.0,26707.0,1190.0,1398.0,1665.0,2756.0,1504.0,3140.0,2770.0,1340.0,1213.0,2080.0,1075.0,1249.0,1803.0,913.0,2076.0,1966.0,1428.0,1124.0,1317.0,1063.0,889.0,566.0,1998.0,1107.0,2075.0,1780.0,1940.0,844.0,1282.0,759.0,3589.0
우리,11814.0,7586.0,6787.0,23540.0,2663.0,4288.0,3897.0,1727.0,0.0,3306.0,3145.0,2732.0,2945.0,2420.0,3771.0,2140.0,1964.0,1541.0,21146.0,1305.0,1420.0,2784.0,1816.0,1864.0,1930.0,1112.0,741.0,1699.0,2295.0,955.0,2175.0,797.0,1481.0,882.0,1722.0,1670.0,1688.0,1228.0,687.0,1291.0,1429.0,1644.0,589.0,913.0,2110.0,1062.0,1543.0,1111.0,1699.0,530.0
말,13218.0,7103.0,8228.0,25661.0,2091.0,5454.0,4063.0,2148.0,3306.0,0.0,2912.0,2332.0,3881.0,1454.0,4577.0,1316.0,2050.0,1317.0,21001.0,830.0,1065.0,2830.0,1850.0,1858.0,1879.0,1592.0,556.0,1574.0,2439.0,620.0,1170.0,728.0,1174.0,614.0,1377.0,1704.0,983.0,1108.0,335.0,1379.0,1416.0,1518.0,370.0,1019.0,1735.0,886.0,1419.0,911.0,1574.0,540.0


In [17]:
#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("../dataset/2022/단어동시출현행렬/7월/공출현빈도_df_7월.csv", encoding='UTF-8')